# ABS Monthly Labour Force 6202

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_data
from abs_summary_plot import plot_summary
from abs_plotting import (
    plot_rows_seas_trend,
    plot_rows_individually,
    plot_rows_collectively,
    iudts_from_row,
)
from plotting import (
    calc_and_plot_growth,
    line_plot,
    plot_covid_recovery,
    state_abbr,
    state_colors,
)

# pandas display settings
pd.options.display.max_rows = 9999
pd.options.display.max_columns = 999

# display all charts in this notebook ...
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_data("6202.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

## Plot

### Labour force summary chart

In [3]:
required = {
    # order from least to most important (ie. bottom to top)
    # chart label: [ABS code, growth over n periods]
    "1 month growth: Civilian pop": ["A84423091W", 1],
    "12 month growth: Civilian pop": ["A84423091W", 12],
    "1 month growth: Num underemployed": ["A85255719L", 1],
    "12 months growth: Num underemployed": ["A85255719L", 12],
    "1 month growth: Num employed": ["A84423043C", 1],
    "12 months growth: Num employed": ["A84423043C", 12],
    "1 month growth: Num unemployed": ["A84423046K", 1],
    "12 months growth: Num unemployed": ["A84423046K", 12],
    "1 month growth: Num hours worked": ["A84426277X", 1],
    "12 month growth: Num hours worked": ["A84426277X", 12],
    "Underemployment ratio": ["A85255724F", 0],
    "Participation rate": ["A84423051C", 0],
    "Employment to pop ratio": ["A84423054K", 0],
    "Unemployment rate": ["A84423050A", 0],
}


plot_summary(
    required,
    abs_dict,
    meta,
    start="2000-01",
    title=f"Key labour force statistics {abs_dict['6202001'].index[-1]}",
    rfooter=source,
    lfooter="Australia. All seasonally adjusted except civilian population. "
    "All values are percentages",
    show=SHOW,
    verbose=False,
)

Overriding x0: True


### Headline seasonally adjusted/trend charts

In [4]:
def headline_charts():
    """Produce headline charts."""

    selector1 = {
        "6202001": mc.table,
        "Persons": mc.did,
    }

    selector2 = {
        "Monthly hours worked in all jobs ;  Persons ;": mc.did,
        "6202019": mc.table,
    }

    for selector in (selector1, selector2):
        plot_rows_seas_trend(
            abs_dict,
            meta,
            selector,
            starts=plot_times,
            tags=plot_tags,
            rfooter=source,
            lfooter="Australia. ",
            verbose=False,
            show=SHOW,
        )


headline_charts()

### Headline COVID recovery charts

In [5]:
def covid_charts():
    """Produce pre/post COVID trajectory charts."""

    service_type = "Seasonally Adjusted"
    selector1 = {
        "Persons": mc.did,
        "6202001": mc.table,
        service_type: mc.stype,
    }
    selector2 = {
        "Monthly hours worked in all jobs ;  Persons ;": mc.did,
        "6202019": mc.table,
        service_type: mc.stype,
    }
    selector3 = {
        "Original": mc.stype,
        "6202001": mc.table,
        "Civilian population aged 15 years and over ;  Persons ;": mc.did,
    }

    for selector in (selector1, selector2, selector3):
        plot_rows_individually(
            abs_dict,
            meta,
            selector=selector,
            plot_function=plot_covid_recovery,
            tags="COVID",
            rfooter=source,
            lfooter="Australia. ",
            show=SHOW,
        )


covid_charts()

### State comparisons 

In [6]:
# use linestyle to help differentiate datasets ...
linestyle = {
    "style": [
        "-.",
        "-",
        "--",
        ":",
    ]
    * 3  # repeat enough to cover all series
}

In [7]:
def state_charts():
    """Produced trend charts for states."""

    states = (
        "New South Wales|Victoria|Queensland|South Australia"
        "|Western Australia|Tasmania|Northern Territory|Australian Capital Territory"
    )
    # service_type = "Seasonally Adjusted"
    service_type = "Trend"
    selector = {
        states: mc.did,
        service_type: mc.stype,
    }
    plot_stems = [
        "Unemployment rate ;  Persons",
        "Participation rate ;  Persons",
    ]

    for stem in plot_stems:
        xselector = selector | {stem: mc.did}
        plot_rows_collectively(
            abs_dict,
            meta,
            selector=xselector,
            regex=True,
            starts=RECENT,
            width=2,
            **linestyle,
            legend={"loc": "best"},
            rfooter=source,
            lfooter=f"Australia. {service_type.capitalize()} series.",
            show=SHOW,
        )


state_charts()

In [8]:
def state_employment_growth():
    """Employed persions compared with the 2019 average / index = 100."""

    states = (
        "New South Wales|Victoria|Queensland|South Australia"
        "|Western Australia|Tasmania|Northern Territory|Australian Capital Territory"
    )
    selector = {
        states: mc.tdesc,
    }
    plot_stems = [
        ["Employed total ;  Persons ;", "Trend"],
        ["Unemployed total ;  Persons ;", "Trend"],
        ["Civilian population aged 15 years and over ;", "Original"],
    ]

    for stem, service_type in plot_stems:
        xselector = {
            **selector,
            stem: mc.did,
            service_type: mc.stype,
        }
        frame = pd.DataFrame()
        for _, row in ra.search_meta(
            meta,
            xselector,
            exact=False,
            regex=True,
            verbose=False,
        ).iterrows():
            series_id, _, _, table, _ = iudts_from_row(row)
            state = row[mc.tdesc].split(" - ")[0]
            state = state.split(", ")[-1]
            frame[state] = abs_dict[table][series_id]
        if len(frame) == 0:
            continue

        for comparison_year in (2019, 2022):
            base = frame.loc[frame.index.year == comparison_year].mean()
            index_df = frame.divide(base) * 100
            index_df = index_df.loc[index_df.index.year >= comparison_year]
            index_df.columns = index_df.columns.map(state_abbr)

            title = f"{stem} cf {comparison_year} Ave.".replace(" ;", ";").replace(
                "  ", " "
            )
            line_plot(
                index_df,
                title=title,
                ylabel=f"Index ({comparison_year} = 100)",
                color=[state_colors[x] for x in index_df.columns],
                axhline={"y": 100, "lw": 0.75, "c": "#555555"},
                legend={"loc": "best", "fontsize": "x-small", "ncols": 2},
                **linestyle,
                rfooter=source,
                lfooter=f"Australia. {service_type.capitalize()} series.",
                width=2,
                show=SHOW,
            )


state_employment_growth()

### Headline Growth

In [9]:
def growth_charts(plot_stems: str, service_type: str):
    """Produce growth charts."""

    plot_rows_individually(
        abs_dict,
        meta,
        # data selection
        selector={
            "6202001": mc.table,
            plot_stems: mc.did,
            service_type: mc.stype,
        },
        regex=True,
        # plotting
        plot_function=calc_and_plot_growth,
        from_=list(plot_times)
        + [(pd.Timestamp.today() - pd.DateOffset(months=16)).strftime("%Y-%m-%d")],
        tag="growth",
        rfooter=source,
        annotate=10,
        lfooter=f"Australia. {service_type.capitalize()} series. ",
        y0=True,
        zero_y=True,
        show=SHOW,
    )


def chart_growth():
    """Structure/select growth charts for plotting."""

    # Seasonally adjusted series
    plot_stems = "|".join(
        [
            "Employed total ;  Persons ;",
            "Unemployed total ;  Persons ;",
            "Labour force total ;  Persons ;",
        ]
    )
    service_type = "Seasonally Adjusted"
    growth_charts(plot_stems, service_type)

    # Original series
    plot_stems = "|".join(
        [
            "Civilian population aged 15 years and over ;  Persons ;",
        ]
    )
    service_type = "Original"
    growth_charts(plot_stems, service_type)


chart_growth()

### Gendered comparisons

In [10]:
def gender_charts():
    """Produce gender comparison charts."""

    genders = ["Males", "Females"]
    colours = ["cornflowerblue", "hotpink"]

    token = "by gender"  # note: also appears in plot title
    plots = {
        # data description                       # table
        f"Employed total ;  > {token} ;": "6202001",
        f"> Employed part-time ;  > {token} ;": "6202001",
        f"> Employed full-time ;  > {token} ;": "6202001",
        f"Unemployed total ;  > {token} ;": "6202001",
        f"Labour force total ;  > {token} ;": "6202001",
        f"Unemployment rate ;  > {token} ;": "6202001",
        f"Participation rate ;  > {token} ;": "6202001",
        f"Monthly hours worked in all jobs ;  > {token} ;": "6202019",
    }

    series_type = "Seasonally Adjusted"

    for plot, table in plots.items():
        # assemble data
        collection = {}
        for gender in genders:
            this_gender = plot.replace(token, gender)

            search = {
                table: mc.table,
                series_type: mc.stype,
                this_gender: mc.did,
            }
            _, ident, units = ra.find_id(meta, search)
            collection[gender] = abs_dict[table][ident]
        df = pd.DataFrame(collection)
        df, units = ra.recalibrate(df, units)

        title = f"Australia: {plot}".replace(" ;", "").replace("  ", " ")
        line_plot(
            df,
            starts=plot_times,
            tags=plot_tags,
            color=colours,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"{series_type.capitalize()} series",
            show=SHOW,
        )


gender_charts()

## Finished

In [11]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Jun 29 2024 20:46:52

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

matplotlib: 3.8.4
readabs   : 0.0.5a0
pandas    : 2.2.2

Watermark: 2.4.3



In [12]:
print("Finished")

Finished
